In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from known.basic import Remap
from tqdm import tqdm
import torch as tt
import torch.nn as nn

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)

default_device = 'cuda' if tt.cuda.is_available() else 'cpu'
factory = dict(device=default_device, dtype=tt.float32)


import deep, maco


In [2]:
import config

In [3]:
tenv = config.tenvF()
venv = config.venvF()
tenv, venv


(<maco.core.WorkersEnv at 0x1aec81de110>,
 <maco.core.WorkersEnv at 0x1aeca253310>)

In [4]:

mean_return, mean_steps, sum_return, sum_steps, acts=deep.rl.Eval.validate_policy(
    envs=[tenv, venv],
    pie=deep.rl.RIE(0, venv.A),
    episodes = 5,
)
mean_return, mean_steps, sum_return, sum_steps, len(acts)

[Test Result]:
            	Total-Episodes	[5]
            	Mean-Reward	[-22.379399999999997]
            	Median-Reward	[-23.242999999999995]
            	Max-Reward	[-17.439]
            	Min-Reward	[-27.351999999999997]
            
[Test Result]:
            	Total-Episodes	[5]
            	Mean-Reward	[-21.839199999999998]
            	Median-Reward	[-21.416999999999998]
            	Max-Reward	[-20.567]
            	Min-Reward	[-24.386]
            


(-22.109299999999998, 60.0, -44.218599999999995, 120.0, 2)

In [5]:

mean_return, mean_steps, sum_return, sum_steps, acts=deep.rl.Eval.validate_policy_once(
    envs=[tenv, venv],
    pie=deep.rl.RIE(0, venv.A), 
    episodic_verbose=1,
)
mean_return, mean_steps, sum_return, sum_steps, len(acts)

[RESET]
[TERM]: Steps:[60], Return:[-20.077]
[RESET]
[TERM]: Steps:[60], Return:[-12.218]


(-16.1475, 60.0, -32.295, 120.0, 2)

In [ ]:
acts

In [6]:
mean_return, mean_steps, test_returns, test_steps, test_actions = deep.rl.Eval.eval_policy(
    env=venv,
    pie=deep.rl.RIE(0, venv.A),
    episodes=5
)
mean_return, mean_steps

[Test Result]:
            	Total-Episodes	[5]
            	Mean-Reward	[-22.010600000000004]
            	Median-Reward	[-21.524]
            	Max-Reward	[-17.648999999999997]
            	Min-Reward	[-29.239]
            


(-22.010600000000004, 60.0)

In [ ]:
test_returns, test_steps, test_actions 

In [7]:
test_returns, test_steps, test_actions  = deep.rl.Eval.test_policy(
    env=venv,
    pie=deep.rl.RIE(0, venv.A),
    verbose=1
)
test_returns, test_steps, test_actions

[RESET]
[TERM]: Steps:[60], Return:[-23.121000000000002]


(-23.121000000000002,
 60,
 [3,
  0,
  0,
  0,
  2,
  0,
  1,
  3,
  3,
  0,
  3,
  3,
  1,
  1,
  1,
  1,
  2,
  0,
  2,
  1,
  1,
  3,
  1,
  0,
  1,
  1,
  2,
  3,
  3,
  2,
  2,
  0,
  3,
  0,
  3,
  0,
  3,
  2,
  2,
  1,
  3,
  3,
  2,
  0,
  0,
  1,
  3,
  2,
  2,
  3,
  2,
  1,
  1,
  1,
  2,
  1,
  0,
  1,
  3,
  0])

In [ ]:
sol = np.random.randint(0, venv.A, size=venv.T)
sol

In [ ]:
test_returns, test_steps, test_actions  = deep.rl.Eval.test_policy(
    env=venv,
    pie=deep.rl.VIE(sol),
    verbose=1
)
test_returns, test_steps, test_actions

In [ ]:
test_returns, test_steps, test_actions  = deep.rl.Eval.test_solution(
    env=venv,
    acts=sol,
    verbose=1
)
test_returns, test_steps, test_actions

In [ ]:
env=venv

rie = deep.rl.RIE(0, env.A)
s, t, d= env.reset()
print(f'{env.obsW=}, {env.pending=}')
cr = 0.0
while not d:
    a = rie.predict()
    s, t, d, r = env.step(a)
    print(f'{env.obsW=}, {env.pending=}, {env.itask=}, {a=}, {t=}, {d=}, {r=}')
    cr+=r
print(f'Return = {cr}')

In [ ]:
env=venv

vie = deep.rl.VIE(np.random.randint(0, env.A, size=env.T))
s, t, d= env.reset()
print(f'{env.obsW=}, {env.pending=}')
cr = 0.0
while not d:
    a = rie.predict(t)
    s, t, d, r = env.step(a)
    print(f'{env.obsW=}, {env.pending=}, {env.itask=}, {a=}, {t=}, {d=}, {r=}')
    cr+=r
print(f'Return = {cr}')

In [ ]:
from maco import fda

env = venv
n_actions = env.A
n_steps =   env.T


#====================================================================================================
#====================================================================================================

def randflowF(): return fda.random_flow(0, n_actions+1, n_steps)
def flow2sol(x): return [ int(abs(i)) % n_actions for i in x ]
def flowcost(x): return deep.rl.Eval.test_cost(env,  flow2sol(x))

#====================================================================================================
#====================================================================================================


#print(f'{placement=}')
top_flows =[(np.zeros(n_steps) + j) for j in range(0, n_actions)] 
   
Flow_X, Flow_fitness  = \
fda.optimize(   MAXITER=    1000,
                randflowF=  randflowF,
                costF=      flowcost,
                beta=       15,
                alpha=      15,
                base_flows= top_flows,
                seed=       None)

for i,(fx, fi) in enumerate(zip(Flow_X, Flow_fitness)):
    print(f'{i=} : {flow2sol(fx)=} : {fi=}')

# select top flows to continue
selected_flows=[]
selected_fitness = []
for flow,fcost in zip(Flow_X, Flow_fitness):
    sol = flow2sol(flow)
    if sol not in selected_flows: 
        selected_flows.append(sol)
        selected_fitness.append(fcost)
selected_n = len(selected_flows)

sorted_cost = np.argsort(selected_fitness)
top_flows = [(np.array(selected_flows[sorted_cost[top]]), selected_fitness[sorted_cost[top]]) for top in range(selected_n)]
#top_costs = [selected_fitness[sorted_cost[top]] for top in range(min(n_top, selected_n))]

print(f'\nSORTED')
for i,(fx, fi) in enumerate(top_flows):
    print(f'{i=} : {fx=} : {fi=}')
    


In [ ]:
env = venv
pie = deep.rl.S2SDQN(
    n_actions = env.A,
    embed_dim = 32,

    encoder_block_size = env.T,
    encoder_vocab_count = env.task_vocab.count,
    encoder_hidden_dim = 64,
    encoder_activation = nn.ReLU(),
    encoder_num_heads = 4,
    encoder_num_layers = 2,
    encoder_norm_eps = 0.0, # final norm # keep zero to not use norm
    encoder_dropout = 0.0,
    encoder_norm_first = True,

    decoder_block_size = env.T,
    decoder_vocab_count = env.worker_vocab.count,
    decoder_hidden_dim = 64,
    decoder_activation = nn.ReLU(),
    decoder_num_heads = 4,
    decoder_num_layers = 2,
    decoder_norm_eps = 0.0, # final norm # keep zero to not use norm
    decoder_dropout = 0.0,
    decoder_norm_first = True,

    dense_layer_dims = [64 ,64, 64],
    dense_actFs = [nn.Tanh(), nn.ReLU() ],
    dense_bias = True,

    xavier_init = False,

    has_target = True, 
    **factory)

deep.modular.count(pie.theta)

In [ ]:
pie.eval()


s, t, d= env.reset()
print(f'{env.obsW=}, {env.pending=}')
cr = 0.0
while not d:
    print(f'\n{t=}')
    a = pie.predict(s, t)
    s, t, d, r = env.step(a)
    print(f'{env.obsW=}, {env.pending=}, {env.itask=}, {a=}, {t=}, {d=}, {r=}')
    cr+=r
print(f'Return = {cr}')

In [ ]:
pie.eval()


s, t, d= env.reset()
print(f'{env.obsW=}, {env.pending=}')
cr = 0.0
while not d:
    print(f'\n{t=}')
    out, outx, a = pie.predict_(s, t)
    print(out.shape, out)
    print(outx.shape, outx)
    s, t, d, r = env.step(a)
    print(f'{env.obsW=}, {env.pending=}, {env.itask=}, {a=}, {t=}, {d=}, {r=}')
    cr+=r
print(f'Return = {cr}')

In [ ]:
pie.eval()

with tt.no_grad():
    s, t, d= env.reset()
    s = tt.stack((s,s))
    print(f'{s.shape=}, {env.obsW=}, {env.pending=}')

    cr = 0.0
    while not d:
        print(f'\n{t=}')
        
        out = pie.forward(s.to(device=default_device))
        print('out', out.shape, out)
        outX = tt.argmax(out, dim=-1)
        print('outX', outX.shape, outX)
        a = outX[:,t]
        print('a', a.shape, a)
        break
        s, t, d, r = env.step(a)
        print(f'{env.obsW=}, {env.pending=}, {env.itask=}, {a=}, {t=}, {d=}, {r=}')
        cr+=r
    print(f'Return = {cr}')

In [ ]:
pie.eval()

with tt.no_grad():
    
    st, tt_, dt= tenv.reset()
    sv, tv_, dv= venv.reset()

    s = tt.stack((st,sv))
    print(f'{s.shape=}')

    d=(dt or dv)

    crt, crv = 0.0, 0.0
    while not (d):
        print(f'\n{tt_=}, {tv_=}')
        
        a= pie.predict_batch(s, tt_)
        #print('a', a.shape, a)
        st, tt_, dt, rt = tenv.step(a[0])
        sv, tv_, dv, rv = venv.step(a[1])
        d=(dt or dv)
        print(f'{a[0]=}, {tt_=}, {dt=}, {rt=}')
        print(f'{a[1]=}, {tv_=}, {dv=}, {rv=}')
        crt+=rt
        crv+=rv
    print(f'Return = {crt}, {crv}')